In [1]:
from optimization.acs import ACS_VRP,ACS_TSP
from optimization.bso import BSO_VRP,BSO_TSP
from optimization.hybrid_acs_bso import ACSBSO_VRP,ACSBSO_TSP
from optimization.ga import GA_VRP
from optimization.sa import SA_VRP
from optimization.tabu_search import TS_VRP
from optimization.dka import DKA_VRP

from optimization.aco import ACO_VRP,ACO_TSP
from optimization.haco import HACO_VRP,HACO_TSP

from optimization.koneksi import ConDB

import sql_connection
import json
import time
import random
import datetime
import copy
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from matplotlib.patches import FancyArrowPatch

import scipy.stats as stats
from scikit_posthocs import posthoc_dunn

import warnings
warnings.filterwarnings('ignore')

# Generate Data

In [2]:
def generate_data(n = 30, random_state = None):
    random.seed(random_state)
    query = """SELECT 
                    p.post_id,
                    p.post_type,
                    CASE
                        WHEN pj.pj_jam_buka = pj.pj_jam_tutup THEN "tutup"
                        ELSE "buka"
                    END AS is_operate
                FROM 
                    posts p
                LEFT JOIN
                    posts_jadwal pj
                    ON p.post_id = pj.pj_id_tempat AND pj.pj_hari = "minggu"
                """

    df_location = sql_connection.read_from_sql(query)
    
    tourid = df_location[(df_location['post_type']=="location")&
                         (df_location['is_operate'] != "tutup")]['post_id'].values.tolist()
    tourid = random.sample(tourid,n)
    idhotel = df_location[df_location['post_type']=="hotel"]['post_id'].values.tolist()
    idhotel = idhotel[random.randint(0,len(idhotel)-1)]
    
    db = ConDB()

    hotel = db.HotelbyID(idhotel)
    tur = db.WisatabyID(tourid)
    timematrix = db.TimeMatrixbyID(hotel._id,tourid)
    
    return hotel,tur,timematrix

# Analisis


In [3]:
def find_node(node_id,tour):
    for node in tour:
        if node._id == node_id:
            return node
    return False

def time_to_second(time):
    return (time.hour*3600)+(time.minute*60)+time.second

def diff_second_between_time(time_a,time_b):
    #input: time_a and time_b, datetime.time()
    #output: time_b - time_a, seconds (int)
    return time_to_second(time_b) - time_to_second(time_a)

def get_attribute_analysis(solution):
    days_needed = len(solution)
    poi_included = len(sum([sol["index"] for sol in solution],[]))
    avg_rating = sum(sum([sol["rating"] for sol in solution],[]))/poi_included
    total_tarif = sum(sum([sol["tarif"] for sol in solution],[]))
    total_duration = sum([diff_second_between_time(j[0],j[-1]) for j in [i['waktu'] for i in solution]])
    return days_needed,poi_included,avg_rating,total_tarif,total_duration

def get_analysis(model,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating):
    start = time.time()
    model.set_model(tour=tur,hotel=hotel,timematrix=timematrix,
                    travel_days=travel_days,
                    degree_waktu = dwaktu,degree_tarif = dtarif,degree_rating = drating)
    solution,fitness = model.construct_solution()[-2:]
    end = time.time()
    run_time = end-start
    
    daily_fitness = []
    for day in solution:
        daily_fitness.append(model.MAUT([day],use_penalty=False))
    days_needed,poi_included,avg_rating,total_tarif,total_duration = get_attribute_analysis(solution)
    results = {"solution":[sol["index"] for sol in solution],
               "fitness":fitness,
               "daily_fitness":daily_fitness,
               "days_needed":days_needed,
               "poi_included":poi_included,
               "avg_rating":avg_rating,
               "total_tarif":total_tarif,
               "total_duration":total_duration,
               "run_time":run_time}
    
    return results

In [4]:
def get_detail_node_on_solution(solution):
    db = ConDB()
    node_solution = []
    for i in solution:
        nodes = db.WisatabyID(i)
        node_solution.append(nodes)
    return node_solution

def visualize_solution(solution_node,hotel):
    # Sample VRP data
    vrp_data = [[hotel]+sol+[hotel] for sol in solution_node]
    
    # Generate random node coordinates
    node_coords = {node._id: (float(node.long), float(node.lat)) for node in set(node for route in vrp_data for node in route)}  

    # Number of routes (you can calculate this)
    num_routes = len(vrp_data)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12,9))

    # Define the colors for routes using a colormap
    cmap = plt.get_cmap('tab20', num_routes)

    for i, route in enumerate(vrp_data):
        x = [node_coords[node._id][0] for node in route]
        y = [node_coords[node._id][1] for node in route]
        color = cmap(i)  # Get the color for the route
        # Plot the nodes (scatter plot)
        ax.scatter(x, y, c=color, label=f'Route {i + 1}', s=100)

        for j in range(len(x) - 1):
            # Make the arrow heads larger by increasing mutation_scale
            arrow = FancyArrowPatch(
                (x[j], y[j]), (x[j + 1], y[j + 1]),
                arrowstyle='->', mutation_scale=15, color=color
            )
            ax.add_patch(arrow)

    # Set node 159 (start and end point) to black
    start_x, start_y = node_coords[hotel._id]
    ax.scatter(start_x, start_y, c='k', label='Hotel', s=100)

    # Add labels for nodes
    for node, (x, y) in node_coords.items():
        ax.annotate(str(node), (x, y), textcoords="offset points", xytext=(0, 10), ha='center')

    # Add legend
    ax.legend()

    plt.show()



## 50 random nodes

In [41]:
def result_to_xlsx_rd_nodes(result,random_ls,filename):
    df = pd.DataFrame()
    for method in result:
        res_method = result[method]
        df_method = {
            "random_seed" : random_ls,
            "fitness" : [],
            "fitness_1" : [],
            "fitness_2" : [],
            "fitness_3" : [],
            "poi_included" : [],
            "rating" : [],
            "tarif" : [],
            "total_duration" : [],
            "run_time" : [],
        }
        for res in res_method:
            df_method["fitness"].append(res['fitness'])
            df_method["fitness_1"].append(res['daily_fitness'][0])
            df_method["fitness_2"].append(res['daily_fitness'][1])
            df_method["fitness_3"].append(res['daily_fitness'][2])
            df_method["poi_included"].append(res['poi_included'])
            df_method["rating"].append(res['avg_rating'])
            df_method["tarif"].append(res['total_tarif'])
            df_method["total_duration"].append(res['total_duration'])
            df_method["run_time"].append(res['run_time'])
        df_method = pd.DataFrame(df_method)
        df_method['method'] = method
        df = pd.concat([df,df_method])
    df.to_excel(filename,index=False)

### All DOI 1

In [40]:
len(list(range(5,(5*50)+1,5)))

50

In [110]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "acsbso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 1,1,1
    
    print("random state = ",random_ls[i])
    
    acsbso_vrp = ACSBSO_VRP(random_state=100)
    results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acsbso_vrp"].append(results_acsbso_vrp)
    
    ga_vrp = GA_VRP(random_state=100)
    results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ga_vrp"].append(results_ga_vrp)

    ts_vrp = TS_VRP(random_state=100)
    results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ts_vrp"].append(results_ts_vrp)

    sa_vrp = SA_VRP(random_state=100)
    results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["sa_vrp"].append(results_sa_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [111]:
random_node_analysis

{'acsbso_vrp': [{'solution': [[90, 52, 99, 66, 55, 93],
    [14, 10, 9, 4, 88, 35, 7, 50, 15, 74],
    [34, 51, 26, 2, 22]],
   'fitness': 0.681778060630262,
   'daily_fitness': [0.6370435883997208,
    0.7053808525506637,
    0.6376004542278128],
   'days_needed': 3,
   'poi_included': 21,
   'avg_rating': 4.604761904761905,
   'total_tarif': 37000,
   'total_duration': 120420,
   'run_time': 10.714658737182617},
  {'solution': [[6, 36, 10, 89, 18, 71, 49],
    [44, 99, 12, 2, 32, 46],
    [34, 39, 74, 66, 57, 22, 21]],
   'fitness': 0.6486284096289016,
   'daily_fitness': [0.5999805391674861,
    0.6423150196656835,
    0.6146307416772019],
   'days_needed': 3,
   'poi_included': 20,
   'avg_rating': 4.574999999999999,
   'total_tarif': 27000,
   'total_duration': 114240,
   'run_time': 25.829691648483276},
  {'solution': [[8, 39, 50, 81, 32, 53, 2],
    [74, 68, 3, 15, 46, 22, 33, 57],
    [20, 85, 71, 93, 43, 36]],
   'fitness': 0.6142148650184502,
   'daily_fitness': [0.5960650447

In [121]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"50_random_nodes_alldoi_1_vrp.xlsx")

In [42]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "acsbso_vrp": [],
    "acs_vrp": [],
    "bso_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 1,1,1
    
    print("random state = ",random_ls[i])
    
    acsbso_vrp = ACSBSO_VRP(random_state=100)
    results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acsbso_vrp"].append(results_acsbso_vrp)
    
    acs_vrp = ACS_VRP(random_state=100)
    results_acs_vrp = get_analysis(acs_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acs_vrp"].append(results_acs_vrp)

    bso_vrp = BSO_VRP(random_state=100)
    results_bso_vrp = get_analysis(bso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["bso_vrp"].append(results_bso_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [43]:
random_node_analysis

{'acsbso_vrp': [{'solution': [[90, 52, 99, 66, 55, 93],
    [14, 10, 9, 4, 88, 35, 7, 50, 15, 74],
    [34, 51, 26, 2, 22]],
   'fitness': 0.681778060630262,
   'daily_fitness': [0.6370435883997208,
    0.7053808525506637,
    0.6376004542278128],
   'days_needed': 3,
   'poi_included': 21,
   'avg_rating': 4.604761904761905,
   'total_tarif': 37000,
   'total_duration': 120420,
   'run_time': 11.111060380935669},
  {'solution': [[6, 36, 10, 89, 18, 71, 49],
    [44, 99, 12, 2, 32, 46],
    [34, 39, 74, 66, 57, 22, 21]],
   'fitness': 0.6486284096289016,
   'daily_fitness': [0.5999805391674861,
    0.6423150196656835,
    0.6146307416772019],
   'days_needed': 3,
   'poi_included': 20,
   'avg_rating': 4.574999999999999,
   'total_tarif': 27000,
   'total_duration': 114240,
   'run_time': 25.619293928146362},
  {'solution': [[8, 39, 50, 81, 32, 53, 2],
    [74, 68, 3, 15, 46, 22, 33, 57],
    [20, 85, 71, 93, 43, 36]],
   'fitness': 0.6142148650184502,
   'daily_fitness': [0.5960650447

In [44]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"50_random_nodes_alldoi_1_vrp_2.xlsx")

In [122]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "acsbso_tsp": [],
    "acs_tsp": [],
    "bso_tsp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 1,1,1
    
    print("random state = ",random_ls[i])
    
    acsbso_tsp = ACSBSO_TSP(random_state=100)
    results_acsbso_tsp = get_analysis(acsbso_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acsbso_tsp"].append(results_acsbso_tsp)
    
    acs_tsp = ACS_TSP(random_state=100)
    results_acs_tsp = get_analysis(acs_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acs_tsp"].append(results_acs_tsp)

    bso_tsp = BSO_TSP(random_state=100)
    results_bso_tsp = get_analysis(bso_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["bso_tsp"].append(results_bso_tsp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [123]:
random_node_analysis

{'acsbso_tsp': [{'solution': [[2, 7, 10, 50, 15, 4, 35, 88, 52, 93],
    [74, 26, 64, 34, 51],
    [9, 48, 14, 90, 30, 66, 83]],
   'fitness': 0.6350734370257433,
   'daily_fitness': [0.701749213836478, 0.6247794374563241, 0.562168064290706],
   'days_needed': 3,
   'poi_included': 22,
   'avg_rating': 4.572727272727273,
   'total_tarif': 117000,
   'total_duration': 121440,
   'run_time': 45.31474709510803},
  {'solution': [[2, 32, 5, 22, 21],
    [46, 12, 51, 29, 74, 57],
    [10, 89, 48, 71, 49, 36, 66]],
   'fitness': 0.569170399504716,
   'daily_fitness': [0.5726839560799736,
    0.5914741068502132,
    0.5767977126937305],
   'days_needed': 3,
   'poi_included': 18,
   'avg_rating': 4.555555555555555,
   'total_tarif': 110500,
   'total_duration': 119760,
   'run_time': 65.03200387954712},
  {'solution': [[53, 2, 32, 48, 43],
    [74, 57, 38, 31, 64, 3, 36],
    [50, 46, 22, 33, 85, 20, 42]],
   'fitness': 0.5397578187772699,
   'daily_fitness': [0.5463423810259691, 0.52441498543

In [124]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"50_random_nodes_alldoi_1_tsp.xlsx")

### Travel Duration 1, other 0

In [7]:
random_ls = [10,20,30,40,50,60,70,80,90,100]
random_node_analysis ={
    "acsbso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
    "dka_vrp": []
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i]) #random state 50 blacklist
    travel_days = 3
    dwaktu,dtarif,drating = 1,0,0
    
    print("random state = ",random_ls[i])
    
    acsbso_vrp = ACSBSO_VRP(random_state=100)
    results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acsbso_vrp"].append(results_acsbso_vrp)
    
    ga_vrp = GA_VRP(random_state=100)
    results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ga_vrp"].append(results_ga_vrp)

    ts_vrp = TS_VRP(random_state=100)
    results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ts_vrp"].append(results_ts_vrp)

    sa_vrp = SA_VRP(random_state=100)
    results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["sa_vrp"].append(results_sa_vrp)
    
    dka_vrp = DKA_VRP(random_state=100)
    results_dka_vrp = get_analysis(dka_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["dka_vrp"].append(results_dka_vrp)

random state =  10
random state =  20
random state =  30
random state =  40
random state =  50
random state =  60
random state =  70
random state =  80
random state =  90
random state =  100


In [8]:
random_node_analysis

{'acsbso_vrp': [{'solution': [[44, 21, 49, 71, 48, 89, 10, 32],
    [6, 12, 22, 97, 74, 2, 36],
    [57, 80, 66, 67, 99, 18, 5, 46]],
   'fitness': 0.6446759259259259,
   'daily_fitness': [0.4740740740740741,
    0.46412037037037035,
    0.47916666666666663],
   'days_needed': 3,
   'poi_included': 23,
   'avg_rating': 4.534782608695653,
   'total_tarif': 196000,
   'total_duration': 120780,
   'run_time': 27.763322591781616},
  {'solution': [[55, 72, 44, 86, 88, 4, 17, 10, 61],
    [6, 81, 12, 90, 14, 97, 74, 13],
    [80, 28, 36, 21, 67, 45, 43, 23]],
   'fitness': 0.6719135802469136,
   'daily_fitness': [0.4849537037037037,
    0.47430555555555554,
    0.4731481481481481],
   'days_needed': 3,
   'poi_included': 25,
   'avg_rating': 4.548,
   'total_tarif': 321000,
   'total_duration': 125520,
   'run_time': 42.795228242874146},
  {'solution': [[11, 69, 10, 7, 9, 1, 53, 74, 40],
    [95, 72, 18, 88, 35, 4, 86, 46, 22],
    [76, 47, 16, 54, 71, 93, 73]],
   'fitness': 0.6701388888888

### Tarif 1, other 0

In [125]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "acsbso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i])
    travel_days = 3
    dwaktu,dtarif,drating = 0,1,0
    
    print("random state = ",random_ls[i])
    
    acsbso_vrp = ACSBSO_VRP(random_state=100)
    results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acsbso_vrp"].append(results_acsbso_vrp)
    
    ga_vrp = GA_VRP(random_state=100)
    results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ga_vrp"].append(results_ga_vrp)

    ts_vrp = TS_VRP(random_state=100)
    results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ts_vrp"].append(results_ts_vrp)

    sa_vrp = SA_VRP(random_state=100)
    results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["sa_vrp"].append(results_sa_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [126]:
random_node_analysis

{'acsbso_vrp': [{'solution': [[14, 99, 7, 15, 2, 35, 51, 9],
    [93, 90, 55, 22, 26, 83],
    [34, 10, 52, 4, 74, 66, 88, 50]],
   'fitness': 0.8128930817610063,
   'daily_fitness': [0.5814465408805032,
    0.5443396226415095,
    0.6333333333333333],
   'days_needed': 3,
   'poi_included': 22,
   'avg_rating': 4.59090909090909,
   'total_tarif': 57000,
   'total_duration': 130440,
   'run_time': 14.06861662864685},
  {'solution': [[38, 34, 10, 74, 66, 57, 21, 36],
    [44, 18, 12, 39, 32, 89, 49],
    [6, 99, 22, 71, 46, 2]],
   'fitness': 0.8186021659565169,
   'daily_fitness': [0.6191740412979351,
    0.6007374631268436,
    0.5681415929203539],
   'days_needed': 3,
   'poi_included': 21,
   'avg_rating': 4.566666666666667,
   'total_tarif': 35000,
   'total_duration': 130320,
   'run_time': 11.186679124832153},
  {'solution': [[46, 99, 33, 85, 57, 93, 43],
    [8, 22, 39, 74, 50, 32, 36],
    [81, 68, 20, 71, 42, 15, 3, 2]],
   'fitness': 0.8063593554147565,
   'daily_fitness': [0

In [127]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"50_random_nodes_dtarif_1_vrp.xlsx")

### Rating 1, other 0

In [128]:
random_ls = list(range(5,(5*50)+1,5))
random_node_analysis ={
    "acsbso_vrp": [],
    "ga_vrp": [],
    "ts_vrp": [],
    "sa_vrp": [],
}
for i in range(len(random_ls)):
    # setting
    hotel,tur,timematrix = generate_data(n=30,random_state=random_ls[i])
    travel_days = 3
    dwaktu,dtarif,drating = 0,0,1
    
    print("random state = ",random_ls[i])
    
    acsbso_vrp = ACSBSO_VRP(random_state=100)
    results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["acsbso_vrp"].append(results_acsbso_vrp)
    
    ga_vrp = GA_VRP(random_state=100)
    results_ga_vrp = get_analysis(ga_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ga_vrp"].append(results_ga_vrp)

    ts_vrp = TS_VRP(random_state=100)
    results_ts_vrp = get_analysis(ts_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["ts_vrp"].append(results_ts_vrp)

    sa_vrp = SA_VRP(random_state=100)
    results_sa_vrp = get_analysis(sa_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
    random_node_analysis["sa_vrp"].append(results_sa_vrp)

random state =  5
random state =  10
random state =  15
random state =  20
random state =  25
random state =  30
random state =  35
random state =  40
random state =  45
random state =  50
random state =  55
random state =  60
random state =  65
random state =  70
random state =  75
random state =  80
random state =  85
random state =  90
random state =  95
random state =  100
random state =  105
random state =  110
random state =  115
random state =  120
random state =  125
random state =  130
random state =  135
random state =  140
random state =  145
random state =  150
random state =  155
random state =  160
random state =  165
random state =  170
random state =  175
random state =  180
random state =  185
random state =  190
random state =  195
random state =  200
random state =  205
random state =  210
random state =  215
random state =  220
random state =  225
random state =  230
random state =  235
random state =  240
random state =  245
random state =  250


In [129]:
random_node_analysis

{'acsbso_vrp': [{'solution': [[88, 50, 52, 22, 35, 2, 7, 93],
    [64, 14, 48, 74, 26, 10, 9, 15],
    [55, 90, 72, 34, 66, 83, 4]],
   'fitness': 0.8275077160493827,
   'daily_fitness': [0.577777777777778,
    0.5222222222222226,
    0.44206349206349216],
   'days_needed': 3,
   'poi_included': 23,
   'avg_rating': 4.6,
   'total_tarif': 169500,
   'total_duration': 129900,
   'run_time': 17.73089838027954},
  {'solution': [[12, 22, 18, 74, 44, 57, 36, 10],
    [39, 64, 51, 71, 49],
    [46, 67, 48, 32, 89, 5, 66, 2, 21]],
   'fitness': 0.7598099046015714,
   'daily_fitness': [0.43333333333333357,
    0.3133333333333333,
    0.4555555555555558],
   'days_needed': 3,
   'poi_included': 22,
   'avg_rating': 4.572727272727273,
   'total_tarif': 141500,
   'total_duration': 132120,
   'run_time': 6.955380916595459},
  {'solution': [[8, 68, 48, 50, 20, 5, 57, 74],
    [46, 2, 81, 3, 93, 53, 36, 15, 85],
    [64, 39, 32, 22, 33, 43]],
   'fitness': 0.751307175446668,
   'daily_fitness': [0.

In [130]:
result_to_xlsx_rd_nodes(random_node_analysis,random_ls,"50_random_nodes_drating_1_vrp.xlsx")

## 87 POIs

In [9]:
def result_to_xlsx(result,n,filename):
    df_result = pd.DataFrame()
    for i in result:
        temp = pd.DataFrame(result[i],columns=["fitness","run_time"])
        temp['n'] = n
        temp['method'] = i
        df_result = pd.concat([df_result,temp])
    df_result.to_excel(filename,index=False)

### All DOIs 1

In [5]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "acsbso_vrp": [],
    "acs_vrp": [],
    "bso_vrp": [],
    "acsbso_tsp": []
}
max_runtime = 100*60 #100 minutes
for i in n:
    # setting
    hotel,tur,timematrix = generate_data(n=i,random_state=30)
    travel_days = 3
    dwaktu,dtarif,drating = 1,1,1
    
    print("n = ",i)
    
    if len(run_time_analysis["acsbso_vrp"])==0 or run_time_analysis["acsbso_vrp"][-1][-1] < max_runtime:
        acsbso_vrp = ACSBSO_VRP(random_state=100)
        results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acsbso_vrp"].append([results_acsbso_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["acs_vrp"])==0 or run_time_analysis["acs_vrp"][-1][-1] < max_runtime:
        acs_vrp = ACS_VRP(random_state=100)
        results_acs_vrp = get_analysis(acs_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acs_vrp"].append([results_acs_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["bso_vrp"])==0 or run_time_analysis["bso_vrp"][-1][-1] < max_runtime:
        bso_vrp = BSO_VRP(random_state=100)
        results_bso_vrp = get_analysis(bso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["bso_vrp"].append([results_bso_vrp[key] for key in ["fitness","run_time"]])
        
    if len(run_time_analysis["acsbso_tsp"])==0 or run_time_analysis["acsbso_tsp"][-1][-1] < max_runtime:
        acsbso_tsp = ACSBSO_TSP(random_state=100)
        results_acsbso_tsp = get_analysis(acsbso_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acsbso_tsp"].append([results_acsbso_tsp[key] for key in ["fitness","run_time"]])

n =  5
n =  10
n =  15
n =  20
n =  25
n =  30
n =  35
n =  40
n =  45
n =  50
n =  55
n =  60
n =  65
n =  70
n =  75
n =  80
n =  87


In [8]:
run_time_analysis

{'acsbso_vrp': [[0.8871090534979426, 0.399127721786499],
  [0.6521678424456201, 3.0734004974365234],
  [0.6153649113966573, 2.4784586429595947],
  [0.6438124632569079, 5.760519981384277],
  [0.7030226563559895, 16.10187029838562],
  [0.6924740386265574, 24.601938247680664],
  [0.6647941396343559, 17.48426342010498],
  [0.6490533283588839, 61.88820123672485],
  [0.6290064506871007, 59.60975694656372],
  [0.614246999528495, 82.09166312217712],
  [0.6055980654362337, 62.79016184806824],
  [0.608312406692506, 111.85149550437927],
  [0.602657584772457, 151.90952253341675],
  [0.587594123970585, 137.89876461029053],
  [0.5891579724634121, 265.48313903808594],
  [0.5864081337421732, 265.683313369751],
  [0.5766677310080689, 303.7056477069855]],
 'acs_vrp': [[0.8871090534979426, 0.14449739456176758],
  [0.6519106407995295, 0.6969480514526367],
  [0.6215377509028303, 0.8635823726654053],
  [0.6493433793270394, 1.1651732921600342],
  [0.6970555781666891, 2.2838354110717773],
  [0.675252404121451

In [10]:
result_to_xlsx(run_time_analysis,n,"3_various_numbers_pois_all_1.xlsx")

### Travel duration 1, other 0

In [11]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "acsbso_vrp": [],
    "acs_vrp": [],
    "bso_vrp": [],
    "acsbso_tsp": []
}
max_runtime = 100*60 #100 minutes
for i in n:
    # setting
    hotel,tur,timematrix = generate_data(n=i,random_state=30)
    travel_days = 3
    dwaktu,dtarif,drating = 1,0,0
    
    print("n = ",i)
    
    if len(run_time_analysis["acsbso_vrp"])==0 or run_time_analysis["acsbso_vrp"][-1][-1] < max_runtime:
        acsbso_vrp = ACSBSO_VRP(random_state=100)
        results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acsbso_vrp"].append([results_acsbso_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["acs_vrp"])==0 or run_time_analysis["acs_vrp"][-1][-1] < max_runtime:
        acs_vrp = ACS_VRP(random_state=100)
        results_acs_vrp = get_analysis(acs_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acs_vrp"].append([results_acs_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["bso_vrp"])==0 or run_time_analysis["bso_vrp"][-1][-1] < max_runtime:
        bso_vrp = BSO_VRP(random_state=100)
        results_bso_vrp = get_analysis(bso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["bso_vrp"].append([results_bso_vrp[key] for key in ["fitness","run_time"]])
        
    if len(run_time_analysis["acsbso_tsp"])==0 or run_time_analysis["acsbso_tsp"][-1][-1] < max_runtime:
        acsbso_tsp = ACSBSO_TSP(random_state=100)
        results_acsbso_tsp = get_analysis(acsbso_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acsbso_tsp"].append([results_acsbso_tsp[key] for key in ["fitness","run_time"]])

n =  5
n =  10
n =  15
n =  20
n =  25
n =  30
n =  35
n =  40
n =  45
n =  50
n =  55
n =  60
n =  65
n =  70
n =  75
n =  80
n =  87


In [12]:
run_time_analysis

{'acsbso_vrp': [[0.9406635802469135, 0.3794410228729248],
  [0.8636574074074074, 2.0229878425598145],
  [0.7993827160493828, 4.102318048477173],
  [0.7623456790123456, 11.819077014923096],
  [0.7101929012345679, 17.788803577423096],
  [0.670138888888889, 34.25940561294556],
  [0.6152447089947091, 25.88098907470703],
  [0.58125, 41.09044790267944],
  [0.5596064814814815, 77.072674036026],
  [0.5329320987654321, 81.13267636299133],
  [0.525003507295174, 116.13746523857117],
  [0.503858024691358, 135.84251952171326],
  [0.4870963912630579, 161.18416595458984],
  [0.44721671075837743, 42.62708854675293],
  [0.47424382716049385, 181.24226117134094],
  [0.46716820987654323, 234.6025996208191],
  [0.4521525117071094, 319.45279836654663]],
 'acs_vrp': [[0.9406635802469135, 0.1155543327331543],
  [0.863503086419753, 0.8237183094024658],
  [0.798148148148148, 1.454474687576294],
  [0.7581018518518519, 2.037566900253296],
  [0.6916203703703704, 2.145169496536255],
  [0.6542438271604938, 5.2704596

In [13]:
result_to_xlsx(run_time_analysis,n,"3_various_numbers_pois_dwaktu_1.xlsx")

### Tarif 1, other 0

In [14]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "acsbso_vrp": [],
    "acs_vrp": [],
    "bso_vrp": [],
    "acsbso_tsp": []
}
max_runtime = 100*60 #100 minutes
for i in n:
    # setting
    hotel,tur,timematrix = generate_data(n=i,random_state=30)
    travel_days = 3
    dwaktu,dtarif,drating = 0,1,0
    
    print("n = ",i)
    
    if len(run_time_analysis["acsbso_vrp"])==0 or run_time_analysis["acsbso_vrp"][-1][-1] < max_runtime:
        acsbso_vrp = ACSBSO_VRP(random_state=100)
        results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acsbso_vrp"].append([results_acsbso_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["acs_vrp"])==0 or run_time_analysis["acs_vrp"][-1][-1] < max_runtime:
        acs_vrp = ACS_VRP(random_state=100)
        results_acs_vrp = get_analysis(acs_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acs_vrp"].append([results_acs_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["bso_vrp"])==0 or run_time_analysis["bso_vrp"][-1][-1] < max_runtime:
        bso_vrp = BSO_VRP(random_state=100)
        results_bso_vrp = get_analysis(bso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["bso_vrp"].append([results_bso_vrp[key] for key in ["fitness","run_time"]])
        
    if len(run_time_analysis["acsbso_tsp"])==0 or run_time_analysis["acsbso_tsp"][-1][-1] < max_runtime:
        acsbso_tsp = ACSBSO_TSP(random_state=100)
        results_acsbso_tsp = get_analysis(acsbso_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acsbso_tsp"].append([results_acsbso_tsp[key] for key in ["fitness","run_time"]])

n =  5
n =  10
n =  15
n =  20
n =  25
n =  30
n =  35
n =  40
n =  45
n =  50
n =  55
n =  60
n =  65
n =  70
n =  75
n =  80
n =  87


In [15]:
run_time_analysis

{'acsbso_vrp': [[0.9902777777777778, 0.3689460754394531],
  [0.75, 1.773472547531128],
  [0.8065972222222222, 4.525331258773804],
  [0.8611166225749559, 3.141202926635742],
  [0.8770612874779542, 11.589081048965454],
  [0.8628094293272864, 20.997997283935547],
  [0.8225769068689424, 16.57103443145752],
  [0.7826664462081129, 19.56944513320923],
  [0.7737700983921285, 57.94050621986389],
  [0.7544646271510517, 82.7585916519165],
  [0.747884708541902, 82.52977085113525],
  [0.7293069417456672, 101.2769980430603],
  [0.6776519870020105, 38.02901649475098],
  [0.7144757298324409, 157.71341943740845],
  [0.6971425786135759, 117.84329152107239],
  [0.6629766949152542, 43.898598432540894],
  [0.6839080459770115, 247.17884182929993]],
 'acs_vrp': [[0.9902777777777778, 0.09415245056152344],
  [0.75, 0.2838101387023926],
  [0.7866898148148149, 2.3702392578125],
  [0.8611937830687831, 1.805323839187622],
  [0.8776785714285715, 2.016840934753418],
  [0.8590986394557824, 2.542783737182617],
  [0.80

In [17]:
result_to_xlsx(run_time_analysis,n,"3_various_numbers_pois_dtarif_1.xlsx")

### Rating 1, other 0

In [18]:
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]
run_time_analysis = {
    "acsbso_vrp": [],
    "acs_vrp": [],
    "bso_vrp": [],
    "acsbso_tsp": []
}
max_runtime = 100*60 #100 minutes
for i in n:
    # setting
    hotel,tur,timematrix = generate_data(n=i,random_state=30)
    travel_days = 3
    dwaktu,dtarif,drating = 0,0,1
    
    print("n = ",i)
    
    if len(run_time_analysis["acsbso_vrp"])==0 or run_time_analysis["acsbso_vrp"][-1][-1] < max_runtime:
        acsbso_vrp = ACSBSO_VRP(random_state=100)
        results_acsbso_vrp = get_analysis(acsbso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acsbso_vrp"].append([results_acsbso_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["acs_vrp"])==0 or run_time_analysis["acs_vrp"][-1][-1] < max_runtime:
        acs_vrp = ACS_VRP(random_state=100)
        results_acs_vrp = get_analysis(acs_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acs_vrp"].append([results_acs_vrp[key] for key in ["fitness","run_time"]])
    
    if len(run_time_analysis["bso_vrp"])==0 or run_time_analysis["bso_vrp"][-1][-1] < max_runtime:
        bso_vrp = BSO_VRP(random_state=100)
        results_bso_vrp = get_analysis(bso_vrp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["bso_vrp"].append([results_bso_vrp[key] for key in ["fitness","run_time"]])
        
    if len(run_time_analysis["acsbso_tsp"])==0 or run_time_analysis["acsbso_tsp"][-1][-1] < max_runtime:
        acsbso_tsp = ACSBSO_TSP(random_state=100)
        results_acsbso_tsp = get_analysis(acsbso_tsp,hotel,tur,timematrix,travel_days,dwaktu,dtarif,drating)
        run_time_analysis["acsbso_tsp"].append([results_acsbso_tsp[key] for key in ["fitness","run_time"]])

n =  5
n =  10
n =  15
n =  20
n =  25
n =  30
n =  35
n =  40
n =  45
n =  50
n =  55
n =  60
n =  65
n =  70
n =  75
n =  80
n =  87


In [19]:
run_time_analysis

{'acsbso_vrp': [[0.8795833333333335, 0.476165771484375],
  [0.8637841710758376, 0.9883723258972168],
  [0.8542603615520283, 4.142903089523315],
  [0.8340225563909776, 6.582834720611572],
  [0.8639510489510489, 9.466446161270142],
  [0.8262820512820512, 17.318251371383667],
  [0.7843336216252883, 22.888553142547607],
  [0.7386900522317187, 22.674569368362427],
  [0.7164036817882968, 38.285372495651245],
  [0.674605887939221, 16.004608631134033],
  [0.6880395466602363, 78.42473554611206],
  [0.6634636582553248, 95.5593192577362],
  [0.6506414349805154, 81.07114362716675],
  [0.6482275788727401, 169.2787516117096],
  [0.6459906759906759, 193.54161262512207],
  [0.6246213774775604, 117.54355430603027],
  [0.6171626186712393, 226.02262139320374]],
 'acs_vrp': [[0.8795833333333335, 0.10995697975158691],
  [0.8638999118165783, 0.4141848087310791],
  [0.8534887566137564, 1.4359488487243652],
  [0.8340225563909776, 0.8985245227813721],
  [0.8613286713286713, 3.403102159500122],
  [0.81309921962

In [21]:
result_to_xlsx(run_time_analysis,n,"3_various_numbers_pois_drating_1.xlsx")

## Uji Statistik

In [15]:
def uji_statistik(first_data,second_data,alternative="greater",first_data_name="first",second_data_name="second"):
    # first_data is the target data, e.g. the results of Hybrid ACS-BSO 
    # second_data is a list of results from other method
    # the test is using Wilcoxon Rank-Sum Test
    second_data_name = ["second" for i in range(len(second_data))] if second_data_name == "second" else second_data_name
    for i in range(len(second_data)):
        statistic, p_value = stats.ranksums(first_data, second_data[i],alternative=alternative)
        print(f"Wilcoxon Rank-Sum Test ({first_data_name} vs. {second_data_name[i]}):")
        print("Test Statistic:", statistic)
        print("p-value:", p_value)
        print("")

### Power Analysis

In [105]:
import numpy as np
from scipy.stats import ranksums

# Parameters for power analysis
alpha = 0.05         # Significance level
effect_size = 0.5    # Desired effect size (e.g., Cohen's d)

# Sample sizes for the two groups
n1 = 50
n2 = 50

# Number of simulations
num_simulations = 1000

# Count of rejections
reject_count = 0

# Perform power analysis through simulation
np.random.seed(42)
for _ in range(num_simulations):
    # Simulate data under the alternative hypothesis with the desired effect size
    group1 = np.random.normal(loc=0, scale=1, size=n1)
    group2 = np.random.normal(loc=effect_size, scale=1, size=n2)
    
    # Perform Wilcoxon rank-sum test
    _, p_value = ranksums(group1, group2)
    
    # Check if null hypothesis is rejected
    if p_value < alpha:
        reject_count += 1

# Calculate power
power = reject_count / num_simulations
print("Power:", power)


Power: 0.7


### General Performance

In [29]:
results_df = pd.read_excel("50_random_nodes_alldoi_1_vrp.xlsx")
results_df.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.681778,0.637044,0.705381,0.637600,21,4.604762,37000,120420,10.714659,acsbso_vrp
1,10,0.648628,0.599981,0.642315,0.614631,20,4.575000,27000,114240,25.829692,acsbso_vrp
2,15,0.614215,0.596065,0.600961,0.537402,21,4.633333,81500,122640,13.316754,acsbso_vrp
3,20,0.680889,0.615864,0.684869,0.638814,22,4.595455,37000,119340,19.968890,acsbso_vrp
4,25,0.634522,0.608162,0.621609,0.616015,20,4.565000,68500,120960,18.717250,acsbso_vrp


In [31]:
results_df.groupby('method').mean()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time
method,,,,,,,,,,
acsbso_vrp,127.5,0.663061,0.627354,0.626240,0.619048,22.28,4.574788,57810.0,119944.8,19.771924
ga_vrp,127.5,0.630304,0.620707,0.602557,0.601438,21.02,4.567250,79670.0,126904.8,0.727035
sa_vrp,127.5,0.655173,0.624463,0.620693,0.613879,21.82,4.573416,54910.0,124672.8,0.944255
ts_vrp,127.5,0.640822,0.620363,0.609971,0.611369,21.32,4.571259,68930.0,126156.0,0.656699


In [18]:
metrics = ["fitness","poi_included","rating","tarif","total_duration"]

for metric in metrics:
    print("metric: ",metric)
    metrics = metric
    alternative = "greater" if metric in ["fitness","poi_included","rating"] else "less"
    first_data = results_df[results_df['method']=="acsbso_vrp"][metrics].values
    first_data_name = "acsbso_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["acsbso_vrp","acsbso_tsp"])]['method'].unique():
        second_data.append(results_df[results_df['method']==method][metrics].values)
        second_data_name.append(method)

    uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

metric:  fitness
Wilcoxon Rank-Sum Test (acsbso_vrp vs. ga_vrp):
Test Statistic: 5.459905341362034
p-value: 2.381942542430321e-08

Wilcoxon Rank-Sum Test (acsbso_vrp vs. ts_vrp):
Test Statistic: 4.191442484277925
p-value: 1.3859323076367846e-05

Wilcoxon Rank-Sum Test (acsbso_vrp vs. sa_vrp):
Test Statistic: 1.8820128259997917
p-value: 0.02991713710157181

metric:  poi_included
Wilcoxon Rank-Sum Test (acsbso_vrp vs. ga_vrp):
Test Statistic: 4.449960729607566
p-value: 4.294299624275932e-06

Wilcoxon Rank-Sum Test (acsbso_vrp vs. ts_vrp):
Test Statistic: 3.4538037576040135
p-value: 0.00027636971661112195

Wilcoxon Rank-Sum Test (acsbso_vrp vs. sa_vrp):
Test Statistic: 1.9509510247543629
p-value: 0.02553143600980818

metric:  rating
Wilcoxon Rank-Sum Test (acsbso_vrp vs. ga_vrp):
Test Statistic: 1.2133122980804518
p-value: 0.1125052213808364

Wilcoxon Rank-Sum Test (acsbso_vrp vs. ts_vrp):
Test Statistic: 0.6032092391024974
p-value: 0.2731847510826574

Wilcoxon Rank-Sum Test (acsbso_vrp v

### Attribute Analysis

#### Travel Duration

In [25]:
n = np.array([5,10,15,20,25,30,35,40])
acsbso = np.array([24540,62580,90840,119940,150480,173400,213180,245760])/n
ga = np.array([24540,64260,95760,129360,161400,190500,229680,273240])/n
ts = np.array([24540,63660,92220,126360,155100,175620,228360,268140])/n
sa = np.array([27120,69900,109680,134640,171420,196500,251820,286980])/n

In [26]:
first_data = acsbso
second_data = [ga,ts,sa]
alternative = 'less'
uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

Wilcoxon Rank-Sum Test (acsbso_vrp vs. ga_vrp):
Test Statistic: -2.573014654401468
p-value: 0.005040846941929832

Wilcoxon Rank-Sum Test (acsbso_vrp vs. ts_vrp):
Test Statistic: -1.7328466039846622
p-value: 0.041561468479951666

Wilcoxon Rank-Sum Test (acsbso_vrp vs. sa_vrp):
Test Statistic: -2.6255251575525183
p-value: 0.004325770766973766



#### Tarif

In [32]:
results_df = pd.read_excel("50_random_nodes_dtarif_1_vrp.xlsx")
results_df.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.812893,0.581447,0.544340,0.633333,22,4.590909,57000,130440,14.068617,acsbso_vrp
1,10,0.818602,0.619174,0.600737,0.568142,21,4.566667,35000,130320,11.186679,acsbso_vrp
2,15,0.806359,0.564511,0.604149,0.577701,22,4.595455,86500,132240,16.528855,acsbso_vrp
3,20,0.847439,0.569773,0.599328,0.642443,23,4.595652,57000,131880,15.483165,acsbso_vrp
4,25,0.791134,0.601364,0.546888,0.600682,20,4.580000,66000,132900,20.183995,acsbso_vrp


In [33]:
results_df.groupby('method').mean()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time
method,,,,,,,,,,
acsbso_vrp,127.5,0.830427,0.604457,0.593020,0.588170,22.56,4.562295,61340.0,131040.0,14.487001
ga_vrp,127.5,0.794451,0.584935,0.573823,0.577585,21.28,4.555451,79340.0,129303.6,0.737736
sa_vrp,127.5,0.823688,0.602343,0.589186,0.586356,22.20,4.560042,61820.0,130809.6,0.949543
ts_vrp,127.5,0.805346,0.595846,0.584381,0.573113,21.62,4.558677,71100.0,128365.2,0.667012


In [28]:
metrics = ["fitness","poi_included","tarif"]

for metric in metrics:
    print("metric: ",metric)
    metrics = metric
    alternative = "greater" if metric in ["fitness","poi_included","rating"] else "less"
    first_data = results_df[results_df['method']=="acsbso_vrp"][metrics].values
    first_data_name = "acsbso_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["acsbso_vrp","acsbso_tsp"])]['method'].unique():
        second_data.append(results_df[results_df['method']==method][metrics].values)
        second_data_name.append(method)

    uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

metric:  fitness
Wilcoxon Rank-Sum Test (acsbso_vrp vs. ga_vrp):
Test Statistic: 6.356101925171458
p-value: 1.034688727511731e-10

Wilcoxon Rank-Sum Test (acsbso_vrp vs. ts_vrp):
Test Statistic: 4.970444130204578
p-value: 3.339985434056978e-07

Wilcoxon Rank-Sum Test (acsbso_vrp vs. sa_vrp):
Test Statistic: 1.3718701552159656
p-value: 0.08505193134758504

metric:  poi_included
Wilcoxon Rank-Sum Test (acsbso_vrp vs. ga_vrp):
Test Statistic: 4.243146133343853
p-value: 1.1020390332711966e-05

Wilcoxon Rank-Sum Test (acsbso_vrp vs. ts_vrp):
Test Statistic: 3.1746040526480006
p-value: 0.0007502054238140426

Wilcoxon Rank-Sum Test (acsbso_vrp vs. sa_vrp):
Test Statistic: 1.2305468477690946
p-value: 0.1092461975870278

metric:  tarif
Wilcoxon Rank-Sum Test (acsbso_vrp vs. ga_vrp):
Test Statistic: -2.9091919874429015
p-value: 0.0018118212857961447

Wilcoxon Rank-Sum Test (acsbso_vrp vs. ts_vrp):
Test Statistic: -1.9785263042561914
p-value: 0.023934682083350964

Wilcoxon Rank-Sum Test (acsbso_v

#### Rating

In [35]:
results_df = pd.read_excel("50_random_nodes_drating_1_vrp.xlsx")
results_df.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.827508,0.577778,0.522222,0.442063,23,4.600000,169500,129900,17.730898,acsbso_vrp
1,10,0.759810,0.433333,0.313333,0.455556,22,4.572727,141500,132120,6.955381,acsbso_vrp
2,15,0.751307,0.374405,0.403968,0.302381,23,4.630435,166500,133860,14.632515,acsbso_vrp
3,20,0.809722,0.539583,0.437037,0.378571,24,4.583333,265000,131700,19.194298,acsbso_vrp
4,25,0.781362,0.428355,0.395887,0.447980,23,4.552174,173500,130560,21.931605,acsbso_vrp


In [36]:
results_df.groupby('method').mean()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time
method,,,,,,,,,,
acsbso_vrp,127.5,0.803109,0.470304,0.449456,0.420703,23.70,4.589172,183390.0,131845.2,15.240533
ga_vrp,127.5,0.776126,0.433778,0.426877,0.428268,22.46,4.571522,192850.0,130124.4,0.693261
sa_vrp,127.5,0.793384,0.445789,0.438588,0.433978,23.32,4.579160,196370.0,131580.0,0.954392
ts_vrp,127.5,0.785149,0.444094,0.429917,0.427619,22.94,4.573304,185080.0,131137.2,0.669594


In [37]:
metrics = ["fitness","poi_included","rating"]

for metric in metrics:
    print("metric: ",metric)
    metrics = metric
    alternative = "greater" if metric in ["fitness","poi_included","rating"] else "less"
    first_data = results_df[results_df['method']=="acsbso_vrp"][metrics].values
    first_data_name = "acsbso_vrp"

    second_data = []
    second_data_name = []
    for method in results_df[~results_df['method'].isin(["acsbso_vrp","acsbso_tsp"])]['method'].unique():
        second_data.append(results_df[results_df['method']==method][metrics].values)
        second_data_name.append(method)

    uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

metric:  fitness
Wilcoxon Rank-Sum Test (acsbso_vrp vs. ga_vrp):
Test Statistic: 3.8329638507541546
p-value: 6.33042821804192e-05

Wilcoxon Rank-Sum Test (acsbso_vrp vs. ts_vrp):
Test Statistic: 2.495562794915475
p-value: 0.006287874720824638

Wilcoxon Rank-Sum Test (acsbso_vrp vs. sa_vrp):
Test Statistic: 1.382210885029151
p-value: 0.08345347942734727

metric:  poi_included
Wilcoxon Rank-Sum Test (acsbso_vrp vs. ga_vrp):
Test Statistic: 4.5154520184244085
p-value: 3.159092580063381e-06

Wilcoxon Rank-Sum Test (acsbso_vrp vs. ts_vrp):
Test Statistic: 2.8850636178788016
p-value: 0.0019566728660068626

Wilcoxon Rank-Sum Test (acsbso_vrp vs. sa_vrp):
Test Statistic: 1.3925516148423367
p-value: 0.08187771159029067

metric:  rating
Wilcoxon Rank-Sum Test (acsbso_vrp vs. ga_vrp):
Test Statistic: 2.7471872203696592
p-value: 0.003005440569544777

Wilcoxon Rank-Sum Test (acsbso_vrp vs. ts_vrp):
Test Statistic: 2.392155496783618
p-value: 0.008374872179099048

Wilcoxon Rank-Sum Test (acsbso_vrp v

### Hybrid vs Standalone

In [38]:
all_1_df = pd.read_excel("3_various_numbers_pois_all_1.xlsx")
# waktu_1_df = pd.read_excel("3_various_numbers_pois_dwaktu_1.xlsx")
# tarif_1_df = pd.read_excel("3_various_numbers_pois_dtarif_1.xlsx")
# rating_1_df = pd.read_excel("3_various_numbers_pois_drating_1.xlsx")

In [39]:
metrics = ["fitness"]

for metric in metrics:
    print("metric: ",metric)
    metrics = metric
    alternative = "greater" if metric in ["fitness","poi_included","rating"] else "less"
    first_data = all_1_df[all_1_df['method']=="acsbso_vrp"][metrics].values
    first_data_name = "acsbso_vrp"

    second_data = []
    second_data_name = []
    for method in all_1_df[~all_1_df['method'].isin(["acsbso_vrp","acsbso_tsp"])]['method'].unique():
        second_data.append(all_1_df[all_1_df['method']==method][metrics].values)
        second_data_name.append(method)

    uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

metric:  fitness
Wilcoxon Rank-Sum Test (acsbso_vrp vs. acs_vrp):
Test Statistic: 0.5510965117827011
p-value: 0.29078375825501845

Wilcoxon Rank-Sum Test (acsbso_vrp vs. bso_vrp):
Test Statistic: 3.49601849662151
p-value: 0.00023612798136090049



### VRP vs TSP

In [46]:
results_vrp = pd.read_excel("50_random_nodes_alldoi_1_vrp_2.xlsx")
results_vrp.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.681778,0.637044,0.705381,0.637600,21,4.604762,37000,120420,11.111060,acsbso_vrp
1,10,0.648628,0.599981,0.642315,0.614631,20,4.575000,27000,114240,25.619294,acsbso_vrp
2,15,0.614215,0.596065,0.600961,0.537402,21,4.633333,81500,122640,13.256881,acsbso_vrp
3,20,0.680889,0.615864,0.684869,0.638814,22,4.595455,37000,119340,20.029216,acsbso_vrp
4,25,0.634522,0.608162,0.621609,0.616015,20,4.565000,68500,120960,18.848519,acsbso_vrp


In [53]:
results_vrp.groupby('method').mean()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time
method,,,,,,,,,,
acs_vrp,127.5,0.655696,0.637008,0.621101,0.605848,21.88,4.581554,59730.0,123607.2,3.802153
acsbso_vrp,127.5,0.663061,0.627354,0.626240,0.619048,22.28,4.574788,57810.0,119944.8,19.697478
bso_vrp,127.5,0.550465,0.566040,0.573433,0.567408,19.56,4.545299,203030.0,123819.6,0.369207


In [47]:
results_tsp = pd.read_excel("50_random_nodes_alldoi_1_tsp.xlsx")
results_tsp.head()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time,method
0,5,0.635073,0.701749,0.624779,0.562168,22,4.572727,117000,121440,45.314747,acsbso_tsp
1,10,0.569170,0.572684,0.591474,0.576798,18,4.555556,110500,119760,65.032004,acsbso_tsp
2,15,0.539758,0.546342,0.524415,0.554215,19,4.605263,163000,130980,45.532017,acsbso_tsp
3,20,0.609275,0.647046,0.632037,0.513842,20,4.555000,116000,125160,16.657934,acsbso_tsp
4,25,0.530500,0.608717,0.577153,0.525740,14,4.507143,148000,114780,58.054875,acsbso_tsp


In [54]:
results_tsp.groupby('method').mean()

,random_seed,fitness,fitness_1,fitness_2,fitness_3,poi_included,rating,tarif,total_duration,run_time
method,,,,,,,,,,
acs_tsp,127.5,0.574796,0.610672,0.589922,0.553933,19.68,4.573806,167790.0,123726.0,10.681849
acsbso_tsp,127.5,0.577889,0.612401,0.597015,0.543610,20.18,4.566720,166910.0,123666.0,43.627122
bso_tsp,127.5,0.536427,0.569848,0.556151,0.568831,18.24,4.542237,200130.0,123444.0,25.650060


In [52]:
metrics = ["fitness",'fitness_1','fitness_2','fitness_3']

first_data_name = results_vrp['method'].unique()
first_data_name.sort()
second_data_name = results_tsp['method'].unique()
second_data_name.sort()

for first_name,second_name in zip(first_data_name,second_data_name):
    for metric in metrics:
        print("metric: ",metric)
        alternative = "greater"
        first_data = results_vrp[results_vrp['method']==first_name][metric].values
        first_data_name = first_name
        
        second_data = [results_tsp[results_tsp['method']==second_name][metric].values]
        second_data_name = [second_name]
        
        uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)
    print("")


# for metric in metrics:
#     print("metric: ",metric)
#     metrics = metric
#     alternative = "greater" if metric in ["fitness","poi_included","rating"] else "less"
#     first_data = all_1_df[all_1_df['method']=="acsbso_vrp"][metrics].values
#     first_data_name = "acsbso_vrp"

#     second_data = []
#     second_data_name = []
#     for method in all_1_df[~all_1_df['method'].isin(["acsbso_vrp","acsbso_tsp"])]['method'].unique():
#         second_data.append(all_1_df[all_1_df['method']==method][metrics].values)
#         second_data_name.append(method)

#     uji_statistik(first_data,second_data,alternative=alternative,first_data_name=first_data_name,second_data_name=second_data_name)

metric:  fitness
Wilcoxon Rank-Sum Test (acs_vrp vs. acs_tsp):
Test Statistic: 8.031300154907536
p-value: 4.822253733904394e-16

metric:  fitness_1
Wilcoxon Rank-Sum Test (acs_vrp vs. acs_tsp):
Test Statistic: 2.5782886334209603
p-value: 0.004964551157280454

metric:  fitness_2
Wilcoxon Rank-Sum Test (acs_vrp vs. acs_tsp):
Test Statistic: 4.039778447017868
p-value: 2.6750858293358874e-05

metric:  fitness_3
Wilcoxon Rank-Sum Test (acs_vrp vs. acs_tsp):
Test Statistic: 4.8670368320727215
p-value: 5.664196139872021e-07


metric:  fitness
Wilcoxon Rank-Sum Test (acsbso_vrp vs. acsbso_tsp):
Test Statistic: 8.272583850548536
p-value: 6.554330467094237e-17

metric:  fitness_1
Wilcoxon Rank-Sum Test (acsbso_vrp vs. acsbso_tsp):
Test Statistic: 1.6889858694869926
p-value: 0.04561106895864911

metric:  fitness_2
Wilcoxon Rank-Sum Test (acsbso_vrp vs. acsbso_tsp):
Test Statistic: 2.874722888065616
p-value: 0.002021911191876548

metric:  fitness_3
Wilcoxon Rank-Sum Test (acsbso_vrp vs. acsbso_tsp

In [34]:
#hana
n = [5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,87]

aco_vrp = [0.8868,0.6493,0.6219,0.6262,0.6734,0.6442,0.6230,0.6081,0.6000,0.5837,0.5849,0.5753,0.5802,0.5718,0.5690,0.5599,0.5571]
haco_vrp = [0.8871,0.6517,0.6248,0.6386,0.6947,0.6726,0.6602,0.6146,0.6022,0.5858,0.5885,0.5792,0.5740,0.5720,0.5727,0.5655,0.5584]

In [35]:
first_data = aco_vrp
second_data = haco_vrp

# Perform the Wilcoxon rank-sum test
statistic, p_value = stats.ranksums(first_data, second_data)
print("Wilcoxon Rank-Sum Test (First vs. Second):")
print("Test Statistic:", statistic)
print("p-value:", p_value)

# Perform the independent t-test
t_statistic, p_value = stats.ttest_ind(first_data, second_data)
print("Independent t-test (First vs. Second):")
print("t-statistic:", t_statistic)
print("p-value:", p_value)


Wilcoxon Rank-Sum Test (First vs. Second):
Test Statistic: -0.46498768181665406
p-value: 0.6419402779493897
Independent t-test (First vs. Second):
t-statistic: -0.28126533618974614
p-value: 0.7803176781786031
